In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(module_path+'/data')

In [2]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from scipy import stats
from scipy.stats import uniform
from sklearn import linear_model, datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from codes.preprocess import *

In [3]:
from codes.preprocess import *
X_train, y_train = load_clean_data('train')

In [4]:
categ_cols = X_train.columns[X_train.dtypes==object].tolist()
numeric_cols = X_train.columns[X_train.dtypes!=object].tolist()
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_transformer, numeric_cols),
        ('categor', categorical_transformer, categ_cols)
    ]
)

In [5]:
pipe = Pipeline([
     ('prep', preprocessor),
     ('estm', RandomForestClassifier())
])

In [9]:

param_grid = dict(estm =[SVC(gamma = 'auto'), LogisticRegression(solver = 'lbfgs',max_iter =1000)],
                  estm__C=[0.1, 10, 100])

#distributions = dict(C=uniform(loc=0, scale=4))
logistic_reg = RandomizedSearchCV(pipe, param_grid, random_state=42)


pipe.fit(X_train, y_train)


Pipeline(memory=None,
         steps=[('prep',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('numeric',
                                                  Pipeline(memory=None,
                                                           steps=[('scaler',
                                                                   StandardScaler(copy=True,
                                                                                  with_mean=True,
                                                                                  with_std=True))],
                                                           verbose=False),
                                                  ['id', 'org_balance',
                                                   'interest_rate', 'ltv',
                                              

In [35]:
logistic = linear_model.LogisticRegression()

In [45]:

# Create regularization hyperparameter distribution using uniform distribution
C = uniform(loc=0, scale=4)

# Create hyperparameter options
hyperparameters = dict(C=C)

In [46]:
clf = RandomizedSearchCV(logistic, hyperparameters, random_state=1, n_iter=100, cv=5, verbose=0, n_jobs= 1)

In [47]:
best_model = clf.fit(X_train, y_train)

/Users/vyju/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:530: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: could not convert string to float: 'N'